In [1]:
%cd ..

c:\Users\joelw\OneDrive\Documents\Projects\Programming\rust-in-python\rust_mod


In [2]:
%pwd

'c:\\Users\\joelw\\OneDrive\\Documents\\Projects\\Programming\\rust-in-python\\rust_mod'

In [3]:
import rust_mod
import timeit
import pandas as pd
import plotly.express as px
from sklearn import linear_model
import numpy as np
import plotly.graph_objects as go
from pysrc import funs as pyfun
import importlib
importlib.reload(pyfun)
importlib.reload(rust_mod)
import pysrc.timer as timer
import numpy as np


In [13]:
from rust_mod import sum_loop
sum_loop(1000)
from pysrc.funs import sum_loop
sum_loop(1000)

0

In [14]:
values = timer.time_function(names, timeit_fun, timeit_setup, [10])

10 out of  10


In [8]:
timeit.timeit("sum_loop(1000)", setup = "from pysrc.funs import sum_loop")

88.53159089997644

In [4]:
names = ['python', 'rust']
timeit_fun = {'python': "sum_loop", 
               'rust': "sum_loop"}
timeit_setup = {'python': "from pysrc.funs import sum_loop", 
               'rust': "from rust_mod import sum_loop"}      
test_input = [100000, 1000000,2000000,4000000,6000000,8000000, 10000000]
num_repeats = 100

In [5]:
values = timer.time_function(names, timeit_fun, timeit_setup, test_input, num_repeats = num_repeats)
params = timer.calc_linear_approximation(names, test_input, values)
fig = timer.plot_time_figure(names, test_input, values, params)

100000 out of  10000000
1000000 out of  10000000
2000000 out of  10000000
4000000 out of  10000000
6000000 out of  10000000
8000000 out of  10000000
10000000 out of  10000000


In [18]:
params = timer.calc_linear_approximation(names, [np.log10(i) for i in test_input], {name: [np.log10(i) for i in values[name]] for name in names} )
fig = timer.plot_time_figure(names, [np.log10(i) for i in test_input], {name: [np.log10(i) for i in values[name]] for name in names} , params)

In [22]:
params = timer.calc_linear_approximation(names, test_input, values)
fig = timer.plot_time_figure(names, test_input, values, params)

In [6]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#55AA55'},
              'mode': 'markers',
              'name': 'python',
              'type': 'scatter',
              'x': [100000, 1000000, 2000000, 4000000, 6000000, 8000000, 10000000],
              'y': [0.011137040999601595, 0.09447224799951073,
                    0.16932968200009782, 0.3308877059997758, 0.5081546920002438,
                    0.6972891449998133, 0.8371729540004162]},
             {'marker': {'color': '#AA5555'},
              'mode': 'markers',
              'name': 'rust',
              'type': 'scatter',
              'x': [100000, 1000000, 2000000, 4000000, 6000000, 8000000, 10000000],
              'y': [0.0008665469998959452, 0.0075066429999424145,
                    0.014280231000157073, 0.027801159000373447,
                    0.043132342000026254, 0.057673321000183936,
                    0.07153015899995807]},
             {'line': {'color': '#55AA55'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'x': [100000, 10000000],
              'y': [0.0034439992552690035, 0.8472817719939754]},
             {'line': {'color': '#AA5555'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'x': [100000, 10000000],
              'y': [2.166714168417433e-05, 0.07160968386475897]},
             {'mode': 'text',
              'showlegend': False,
              'text': 'python: 8.4e-08x+0.0034',
              'type': 'scatter',
              'x': [2080000.0],
              'y': [0.7535423133003643]},
             {'mode': 'text',
              'showlegend': False,
              'text': 'rust: 7.2e-09x+2.2e-05',
              'type': 'scatter',
              'x': [2080000.0],
              'y': [0.6699116726003123]}],
    'layout': {'template': '...'}
})